In [1]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Model,Sequential
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from keras.layers import GRU,Embedding,Bidirectional,InputLayer,Input,Dense,TimeDistributed,Activation,RepeatVector,Bidirectional,Dot,Concatenate,LSTM
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
print(os.listdir("../input"))
from keras.preprocessing.sequence import pad_sequences
from keras.losses import sparse_categorical_crossentropy
from keras.utils.np_utils import to_categorical
# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


['small_vocab_fr.txt', 'small_vocab_en.txt']


In [7]:
with open("../input/small_vocab_en.txt") as f:
    eng=f.read()

In [6]:
with open("../input/small_vocab_fr.txt") as f:
    fr=f.read()

In [8]:
fr=fr.split("\n")

In [9]:
eng=eng.split("\n")

In [10]:
from collections import Counter

In [11]:
engdict=Counter([token for sent in eng for token in sent.split(" ")])

In [12]:
frdict=Counter(token for sent in fr for token in sent.split(" "))

In [13]:
eng_vocabsize=len(engdict.keys())

In [14]:
fr_vocabsize=len(frdict.keys())

In [15]:
#Tokenize words into indexes we use keras tokenizer
engtokenizer=Tokenizer()
engtokenizer.fit_on_texts(eng)

In [16]:
eng_token=engtokenizer.texts_to_sequences(eng)

In [17]:
frtokenizer=Tokenizer()
frtokenizer.fit_on_texts(fr)
fr_token=frtokenizer.texts_to_sequences(fr)

In [18]:
eng_ind=engtokenizer.word_index

In [19]:
fr_ind=frtokenizer.word_index

In [20]:
eng_token_pad=pad_sequences(eng_token,padding='post')

In [21]:
fr_token_pad=pad_sequences(fr_token,padding='post')

In [24]:
eng_token_pad.shape

(137861, 15)

In [23]:
fr_token_pad.shape

(137861, 21)

In [25]:
ind_w=frtokenizer.index_word

In [26]:
ind_w[0]='<PAD>'

In [27]:
max_eng_length=eng_token_pad.shape[1]
max_fr_length=fr_token_pad.shape[1]

In [28]:
word_ind=frtokenizer.word_index

In [29]:
word_ind['<pad>']=0

In [30]:
Tx=max_eng_length
Ty=max_fr_length

In [32]:
densor=Dense(32,activation='relu')
densor1=Dense(1,activation='relu')
repeat=RepeatVector(Tx)
activator=Activation('softmax')
dotor=Dot(axes=1)

In [33]:
def one_step_attention(a,st_prev):
    st_prev1=repeat(st_prev)  # m X Tx X n_s
    concat=Concatenate(axis=-1)([a,st_prev1])
    dens=densor(concat)
    dens1=densor1(dens)  #weights for attention  m X Tx
    context=dotor([dens1,a])
    context=activator(context)
    return context

In [34]:
n_a=128
n_s=128
post_activation_lstm=LSTM(n_s,return_state=True)
outputlayer=Dense(len(word_ind),activation='softmax')

In [35]:
eng_w=engtokenizer.word_index
eng_w['<pad>']=0
len1=len(eng_w)

In [36]:
x=Input(shape=(Tx,))
embed=Embedding(len1,64,input_length=Tx)(x)

In [37]:
s0=Input(shape=(n_s,))
c0=Input(shape=(n_s,))
s=s0
c=c0

In [38]:
outputs=[]
a=LSTM(n_a,return_sequences=True)(embed)

In [39]:
for i in range(Ty):
    context=one_step_attention(a,s)
    s,_,c=post_activation_lstm(context,initial_state=[s,c])
    
    out=outputlayer(s)
    outputs.append(out)
    
model=Model(inputs=[x,s0,c0],outputs=outputs)

In [40]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 15)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 15, 64)       12800       input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 128)          0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, 15, 128)      98816       embedding_1[0][0]                
__________________________________________________________________________________________________
repeat_vec

In [41]:
model.compile(optimizer='adam',metrics=['accuracy'],loss='categorical_crossentropy')

In [42]:
m=len(eng_token_pad)
epochs=5

In [43]:
s0=np.zeros((m,n_s))
c0=np.zeros((m,n_s))


In [44]:
len(word_ind)

346

In [45]:
y_hot=to_categorical(fr_token_pad,len(word_ind))

In [46]:
model.fit([eng_token_pad,s0,c0],list(y_hot.swapaxes(0,1)),epochs=50,batch_size=64)

Epoch 1/50
137861/137861 [==============================] - 364s 3ms/step - loss: 51.9573 - dense_5_loss: 0.0614 - dense_5_acc: 0.0787 - dense_5_acc_1: 0.2270 - dense_5_acc_2: 0.3306 - dense_5_acc_3: 0.1150 - dense_5_acc_4: 0.0874 - dense_5_acc_5: 0.1023 - dense_5_acc_6: 0.1503 - dense_5_acc_7: 0.1509 - dense_5_acc_8: 0.1653 - dense_5_acc_9: 0.2197 - dense_5_acc_10: 0.2575 - dense_5_acc_11: 0.2857 - dense_5_acc_12: 0.3847 - dense_5_acc_13: 0.5306 - dense_5_acc_14: 0.7614 - dense_5_acc_15: 0.9153 - dense_5_acc_16: 0.9734 - dense_5_acc_17: 0.9938 - dense_5_acc_18: 0.9987 - dense_5_acc_19: 0.9995 - dense_5_acc_20: 0.9995
Epoch 2/50
137861/137861 [==============================] - 344s 2ms/step - loss: 36.5372 - dense_5_loss: 0.0015 - dense_5_acc: 0.1844 - dense_5_acc_1: 0.3649 - dense_5_acc_2: 0.4631 - dense_5_acc_3: 0.2124 - dense_5_acc_4: 0.2770 - dense_5_acc_5: 0.2499 - dense_5_acc_6: 0.3366 - dense_5_acc_7: 0.3911 - dense_5_acc_8: 0.3918 - dense_5_acc_9: 0.4464 - dense_5_acc_10: 0.419

137861/137861 [==============================] - 340s 2ms/step - loss: 3.6128 - dense_5_loss: 1.4989e-04 - dense_5_acc: 0.8341 - dense_5_acc_1: 0.9637 - dense_5_acc_2: 0.9393 - dense_5_acc_3: 0.8802 - dense_5_acc_4: 0.8730 - dense_5_acc_5: 0.9068 - dense_5_acc_6: 0.9264 - dense_5_acc_7: 0.9364 - dense_5_acc_8: 0.9437 - dense_5_acc_9: 0.9540 - dense_5_acc_10: 0.9527 - dense_5_acc_11: 0.9544 - dense_5_acc_12: 0.9587 - dense_5_acc_13: 0.9659 - dense_5_acc_14: 0.9757 - dense_5_acc_15: 0.9874 - dense_5_acc_16: 0.9946 - dense_5_acc_17: 0.9987 - dense_5_acc_18: 0.9998 - dense_5_acc_19: 1.0000 - dense_5_acc_20: 1.0000
Epoch 15/50
137861/137861 [==============================] - 340s 2ms/step - loss: 3.2943 - dense_5_loss: 1.3759e-04 - dense_5_acc: 0.8629 - dense_5_acc_1: 0.9652 - dense_5_acc_2: 0.9448 - dense_5_acc_3: 0.8943 - dense_5_acc_4: 0.8899 - dense_5_acc_5: 0.9152 - dense_5_acc_6: 0.9311 - dense_5_acc_7: 0.9403 - dense_5_acc_8: 0.9474 - dense_5_acc_9: 0.9555 - dense_5_acc_10: 0.9550 - 

137861/137861 [==============================] - 331s 2ms/step - loss: 1.5549 - dense_5_loss: 1.3015e-04 - dense_5_acc: 0.9827 - dense_5_acc_1: 0.9843 - dense_5_acc_2: 0.9753 - dense_5_acc_3: 0.9514 - dense_5_acc_4: 0.9443 - dense_5_acc_5: 0.9527 - dense_5_acc_6: 0.9617 - dense_5_acc_7: 0.9648 - dense_5_acc_8: 0.9692 - dense_5_acc_9: 0.9735 - dense_5_acc_10: 0.9740 - dense_5_acc_11: 0.9762 - dense_5_acc_12: 0.9784 - dense_5_acc_13: 0.9820 - dense_5_acc_14: 0.9864 - dense_5_acc_15: 0.9927 - dense_5_acc_16: 0.9969 - dense_5_acc_17: 0.9991 - dense_5_acc_18: 0.9998 - dense_5_acc_19: 1.0000 - dense_5_acc_20: 1.0000
Epoch 28/50
137861/137861 [==============================] - 329s 2ms/step - loss: 1.4764 - dense_5_loss: 1.2796e-04 - dense_5_acc: 0.9838 - dense_5_acc_1: 0.9851 - dense_5_acc_2: 0.9757 - dense_5_acc_3: 0.9537 - dense_5_acc_4: 0.9462 - dense_5_acc_5: 0.9547 - dense_5_acc_6: 0.9634 - dense_5_acc_7: 0.9671 - dense_5_acc_8: 0.9710 - dense_5_acc_9: 0.9745 - dense_5_acc_10: 0.9751 - 

137861/137861 [==============================] - 318s 2ms/step - loss: 1.0762 - dense_5_loss: 1.3032e-04 - dense_5_acc: 0.9917 - dense_5_acc_1: 0.9904 - dense_5_acc_2: 0.9833 - dense_5_acc_3: 0.9635 - dense_5_acc_4: 0.9576 - dense_5_acc_5: 0.9650 - dense_5_acc_6: 0.9719 - dense_5_acc_7: 0.9752 - dense_5_acc_8: 0.9785 - dense_5_acc_9: 0.9807 - dense_5_acc_10: 0.9812 - dense_5_acc_11: 0.9829 - dense_5_acc_12: 0.9839 - dense_5_acc_13: 0.9865 - dense_5_acc_14: 0.9901 - dense_5_acc_15: 0.9948 - dense_5_acc_16: 0.9977 - dense_5_acc_17: 0.9994 - dense_5_acc_18: 0.9999 - dense_5_acc_19: 1.0000 - dense_5_acc_20: 1.0000
Epoch 41/50
137861/137861 [==============================] - 335s 2ms/step - loss: 1.0769 - dense_5_loss: 1.2543e-04 - dense_5_acc: 0.9903 - dense_5_acc_1: 0.9899 - dense_5_acc_2: 0.9825 - dense_5_acc_3: 0.9627 - dense_5_acc_4: 0.9578 - dense_5_acc_5: 0.9646 - dense_5_acc_6: 0.9714 - dense_5_acc_7: 0.9749 - dense_5_acc_8: 0.9783 - dense_5_acc_9: 0.9807 - dense_5_acc_10: 0.9815 - 

In [ ]:
# While training models on machine translation on different attention models
# Scalar product Attention gives better accuracy in from the starting but it is slow as compared to attention applied
# in this notebook

In [47]:
pred=model.predict([eng_token_pad,s0,c0])

In [53]:
a1=np.ones((10,5))
a2=np.ones((10,5))
a3=np.ones((10,5))


In [59]:
np.stack([a1,a2,a3],axis=1).shape

(10, 3, 5)

In [60]:
y_hot.shape

(137861, 21, 346)

In [61]:
pred1=[np.argmax(s,axis=-1) for s in pred]

In [65]:
pred1=np.stack(pred1,axis=1)

In [66]:
pred1.shape

(137861, 21)

In [67]:
fr_token_pad.shape

(137861, 21)

In [68]:
from nltk.translate.bleu_score import sentence_bleu

In [71]:
bleuscore=[]
for k in range(len(fr_token_pad)):
    x1=[ind_w[i] for i in pred1[k]]
    y1=[ind_w[j] for j in fr_token_pad[k]]
    score=sentence_bleu([x1],y1,weights=(0.25,0.25,0.25,0.25))
    bleuscore.append(score)

In [73]:
np.mean(bleuscore)

0.9784636380978056